In [1]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option("display.float_format", lambda x: f"{x :.2f}")

In [3]:
con = duckdb.connect('movies.db')

In [4]:
df = con.table('movies').df()
con.close()

In [5]:
df.head()

,backdrop_path,id,imdb_id,original_title,overview,popularity,poster_path,release_date,runtime,status,tagline,title,vote_average,vote_count,genres,actors,actors_id,directors,directors_id,video_name,video_key,keywords
0,/qYHM1QaR86eQeKX8qi8kFtqFAGd.jpg,655363,tt9834506,Post Mortem,"Hongrie, 1919. Miraculeusement revenu du front...",18.31,/4X47HT16UXf96cn1pI2hq31tX31.jpg,2020-10-28,115,Released,Immortalisez vos plus grandes peurs.,Post Mortem,6.70,193,"[Horreur, Mystère, Thriller]","[Viktor Klem, Fruzsina Hais, Judit Schell, Ang...","[1739471, 2481242, 125682, 131215, 1499191, 89...","[Péter Bergendy, Péter Bergendy]","[930499, 930499]",[POST MORTEM Bande Annonce VF (2022)],[pv4NBtvy4zw],[]
1,/dhr0q4eiRr8ltqPig32TwhPRdaD.jpg,17,tt0411267,The Dark,En débarquant dans cette contrée isolée du pay...,32.94,/ebzumprgCpfFUZOTJNZ407FA9IH.jpg,2005-09-28,87,Released,,The Dark,5.80,256,"[Horreur, Thriller, Mystère]","[Maria Bello, Sean Bean, Abigail Stone, Richar...","[49, 48, 55, 51, 54, 56]","[John Fawcett, Paul W. S. Anderson, Elizabeth ...","[43, 4014, 1340126, 3973, 2243786, 2248464, 23...",[],[],"[sea, wales, child abuse, shepherd, adolescenc..."
2,/bQ8fRUaitJvi54O2lUT6Ta7FVHK.jpg,25,tt0418763,Jarhead,"Été 1990. Anthony Swofford, fils et petit‐fils...",93.34,/rpPa7dGWnuldRGyLtr4eT2n38jH.jpg,2005-11-04,123,Released,"La guerre, c’est l’enfer. L’attente, c’est pire.",Jarhead : La Fin de l'innocence,6.65,2747,"[Drame, Guerre]","[Jake Gyllenhaal, Jamie Foxx, Peter Sarsgaard,...","[131, 134, 133, 132, 2955, 10964]","[Sam Mendes, Alan B. Curtiss, Luke Longin, Jay...","[39, 62543, 1957599, 1342669, 2315352, 2239554...",[Jarhead: La fin de l'innocence (VF) - Bande A...,"[3wZjh-2PbIM, peoCGkXCCp4]","[california, gulf war, saudi arabia, petrol, m..."
3,/pR12U3MPxq6aeHHd4sQOml72SSH.jpg,98339,tt1867093,The Samaritan,"Foley, un ancien arnaqueur, tente de changer d...",11.83,/5Lvo5OjSZmirakT8B7xxEnn1h1F.jpg,2012-03-02,90,Released,,Le Samaritain,5.40,197,[Thriller],"[Samuel L. Jackson, Luke Kirby, Ruth Negga, To...","[2231, 7002, 17018, 207, 52703, 6575]","[David Weaver, David Weaver, David Weaver, Jil...","[545229, 545229, 545229, 1381934, 1124948]",[],[],"[prison, drug use, revenge, incest, grifter, p..."
4,/8ln5VUVk7gnNGIb2VNgw8nICDVt.jpg,35,tt0462538,The Simpsons Movie,Lorsque Homer pollue gravement le lac de Sprin...,81.37,/9Vz5sgIG10YNka14cv11dO5CfMF.jpg,2007-07-25,88,Released,La plus grande aventure familiale de tous les ...,"Les Simpson, le film",7.01,7711,"[Animation, Comédie, Familial]","[Dan Castellaneta, Julie Kavner, Nancy Cartwri...","[198, 199, 200, 5586, 5587, 6008]","[David Silverman, Matthew Schofield, Juan 'Jo'...","[197, 1447342, 1460481, 1447450]","[LES SIMPSON - LE FILM (bande-annonce), Interv...","[8arbBxezySc, Ji32yfWuROQ]","[garbage, pollution, quarantine, environmental..."


In [19]:
import json
import pandas as pd
import duckdb
"""
Connect to DuckDB in-memory database
conn = duckdb.connect(':memory:')

Register DataFrame as DuckDB table
conn.register('my_table', df)

Query the table
result = conn.execute("SELECT * FROM my_table").fetchall()
print(result)

Close connection
conn.close()
"""
#Sample nested JSON data
nested_json_data = {
    "id": 1,
    "name": "John",
    "details": {'crew':[{
        "age": 30,
        "address": "123 Main St",
        "email": "john@example.com"},
        {"age": 30,
        "address": "123 Main St",
        "email": "john@example.com"}
    ]
    }
}

#Parse JSON data
df = pd.json_normalize(nested_json_data)
df.head()


,id,name,details.crew
0,1,John,"[{'age': 30, 'address': '123 Main St', 'email'..."
